# PHYS20762 - Project - 

David Phelan  
University of Manchester  
March 2025

This project aims to simulate a driven, damped harmonic oscillator.

In [31]:
# Initilisation
import numpy as np
import matplotlib.pyplot as plt

The below function is an Euler solution to the equation of motion:

$m \frac{d^2 x(t)}{dt^2} + b \frac{d x(t)}{dt} + kx(t) = F(t),$

where the external force, $F(t)$ is $0$ (not driven) and accepting m, b, and k as inputs.

In [ ]:
def Euler_undriven(m, k, b, n, x0, v0, dt):
    """
    Forward Euler method for the undriven (possibly damped) oscillator:
        m x''(t) + b x'(t) + k x(t) = 0

    Args:
        m   (float): Mass of the oscillator.
        k   (float): Spring constant.
        b   (float): Damping coefficient (0 for no damping).
        n   (int)  : Number of time steps.
        x0  (float): Initial position x(0).
        v0  (float): Initial velocity x'(0).
        dt  (float): Timestep size.

    Returns:
        x (ndarray): Positions at each timestep (length n+1).
        v (ndarray): Velocities at each timestep (length n+1).
        a (ndarray): Accelerations at each timestep (length n+1).
        t (ndarray): Array of time points from 0 to n*dt (length n+1).

    Notes:
        - This function implements the basic "forward Euler" scheme.
        - The acceleration is computed via a[i+1] = -(k*x[i+1] + b*v[i+1]) / m
          before x[i+1], v[i+1] are updated, matching your original code flow.
        - For purely undamped motion, pass b=0.
    """
    
    # Create an array of time points
    t = np.linspace(0, n*dt, n+1)
    
    # Allocate arrays for position, velocity, and acceleration
    x = np.zeros(n+1)
    v = np.zeros(n+1)
    a = np.zeros(n+1)
    
    # Initial conditions
    x[0] = x0
    v[0] = v0
    a[0] = -(k * x[0] + b * v[0]) / m # Compute initial acceleration at t=0
    
    # Forward Euler time-stepping
    for i in range(n):
        # Compute acceleration for the next index (as in your original code)
        a[i+1] = -(k * x[i+1] + b * v[i+1]) / m
        
        # Update velocity using the old acceleration
        v[i+1] = v[i] + a[i] * dt
        
        # Update position using the old velocity
        x[i+1] = x[i] + v[i] * dt
    
    return x, v, a, t


Below is code to verify that the function is performing as expected, by checking generated values with expected values using the python keyword assert.

In [ ]:
def test_Euler_undriven():
    """
    Tests the Euler_undriven function in simple edge cases
    to confirm that the solver behaves as expected.
    
    Scenario 1: 
        - m=1, k=0, b=0, x0=0, v0=0, dt=0.1
        - Expect no forces and no initial motion => everything remains zero.
    
    Scenario 2:
        - m=1, k=0, b=0, x0=2, v0=-1, dt=0.1
        - Expect velocity remains constant at -1, 
          position decreases linearly with slope -1, 
          acceleration is always zero.
    """
    # -- Scenario 1: no forces and no initial velocity --
    from_module = Euler_undriven(m=1, k=0, b=0, x0=0, v0=0, dt=0.1, n=10)
    x_arr, v_arr, a_arr, t_arr = from_module
    
    # Our expected arrays: everything should be zero (length n_stneps+1 = 11)
    zeros_11 = np.zeros(11)
    
    # Assert they match (within floating-point tolerance)
    assert np.allclose(x_arr, zeros_11), "Scenario 1: x should remain zero."
    assert np.allclose(v_arr, zeros_11), "Scenario 1: v should remain zero."
    assert np.allclose(a_arr, zeros_11), "Scenario 1: a should remain zero."
    
    # -- Scenario 2: constant velocity, no forces --
    from_module = Euler_undriven(m=1, k=0, b=0, x0=2, v0=-1, dt=0.1, n=10)
    x_arr, v_arr, a_arr, t_arr = from_module
    
    # Time array for reference
    # t_arr = [0.0, 0.1, 0.2, ..., 1.0] (length = 11)
    
    # The velocity should remain at -1:
    expected_v = -1 * np.ones_like(t_arr)
    # The position should be x(t) = 2 + v0 * t = 2 - 1 * t
    expected_x = 2 - t_arr
    # Acceleration should be zero (no spring, no damping)
    expected_a = np.zeros_like(t_arr)
    
    assert np.allclose(v_arr, expected_v),  "Scenario 2: velocity should stay at -1."
    assert np.allclose(x_arr, expected_x),  "Scenario 2: position should be linear in time."
    assert np.allclose(a_arr, expected_a),  "Scenario 2: acceleration should remain zero."
    
    print("All Euler_undriven tests passed successfully!")

test_Euler_undriven()

All Euler_undriven tests passed successfully!


The below function is an improved Euler solution to the equation of motion:

$m \frac{d^2 x(t)}{dt^2} + b \frac{d x(t)}{dt} + kx(t) = F(t),$

where the external force, $F(t)$ is $0$ (not driven) and accepting m, b, and k as inputs.

In [ ]:
def ImprovedEuler_undriven(m, k, b, n, x0, v0, dt):
    """
    Implements the 'Improved Euler' approach shown in the slide:
        x_{i+1} = x_i + h * v_i + 0.5 * h^2 * a_i
        v_{i+1} = v_i + h * a_i
        a_i     = -(k/m)*x_i - (b/m)*v_i

    Args:
        m   : mass
        k   : spring constant
        b   : damping coefficient
        n   : number of time steps
        x0  : initial position
        v0  : initial velocity
        dt  : time step

    Returns:
        x, v, a, t
          x, v, a are arrays of length (n+1);
          t is the array of time points from 0 to n*dt.
    """

    # Create an array of time points
    t = np.linspace(0, n*dt, n+1)
    
    # Allocate arrays for position, velocity, and acceleration
    x = np.zeros(n+1)
    v = np.zeros(n+1)
    a = np.zeros(n+1)

    # Initial conditions
    x[0] = x0
    v[0] = v0
    a[0] = -(k*x[0] + b*v[0]) / m  # acceleration at i=0

    for i in range(n):
        # Position update includes a half-step for the acceleration
        x[i+1] = x[i] + dt*v[i] + 0.5*(dt**2)*a[i]
        
        # Velocity update is the standard Euler step
        v[i+1] = v[i] + dt*a[i]

        # Compute the new acceleration at i+1
        a[i+1] = -(k*x[i+1] + b*v[i+1]) / m

    return x, v, a, t